In [2]:
import re
import os
import numpy as np
from progress.bar import Bar
from tqdm import tqdm


In [3]:

def writeRadius(r1,r2):
    lpMesh = open("loop.geo", "r")
    loopMesh = lpMesh.read()
    lpMesh.close()

    reg1 = r"(%r1)"
    reg2 = r"(%r2)"

    sub1 = str(r1)
    sub2 = str(r2)

    loopMesh = re.sub(reg1,sub1,loopMesh,0,re.MULTILINE)
    loopMesh = re.sub(reg2,sub2,loopMesh,0,re.MULTILINE)

    try:
        lpTemp = open("loopTemp.geo", "x")
    except:
        os.remove("loopTemp.geo")
        lpTemp = open("loopTemp.geo","x")

    lpTemp.write(loopMesh)
    lpTemp.close()


    # write surface mesh
    lpMesh = open("loopFIELD.geo", "r")
    loopMesh = lpMesh.read()
    lpMesh.close()
    reg1 = r"(%r1)"
    reg2 = r"(%r2)"
    sub1 = str(r1)
    sub2 = str(r2)
    loopMesh = re.sub(reg1,sub1,loopMesh,0,re.MULTILINE)
    loopMesh = re.sub(reg2,sub2,loopMesh,0,re.MULTILINE)
    try:
        lpTemp = open("loopFIELDTemp.geo", "x")
    except:
        os.remove("loopFIELDTemp.geo")
        lpTemp = open("loopFIELDTemp.geo","x")
    lpTemp.write(loopMesh)
    lpTemp.close()

def runGMSH():
    print(f"{os.getpid()}: Meshing geometry\n")
    # os.system("gmsh loopFIELDTemp.geo -nopopup -0 -v 1")
    # os.system("gmsh loopTemp.geo -nopopup -0 -v 1")
    os.popen("gmsh loopFIELDTemp.geo -nopopup -0 -v 1").read()
    os.popen("gmsh loopTemp.geo -nopopup -0 -v 1").read()
    try:
        os.system("rm *.geo_unrolled")
        os.remove("loopFIELDTemp.geo")
        os.remove("loopTemp.geo")
    except:
        print("nope")
    print(f"{os.getpid()}: Done meshing\n")


def runTTH():
    print(f"{os.getpid()}: Running tth settings_washer.json >/dev/null 2>&1")
    os.system("tth settings_washer.json >/dev/null 2>&1")
    print(f"{os.getpid()}: Done\n")


def getCurrent(path):
    iFile = open(path)
    it = iFile.read()
    iFile.close()
    reg = r"fone fone .+ (\d*\.\d*e.\d*)"
    i_s = re.findall(reg,it,re.MULTILINE)[0]
    return i_s
    
def runNoiseEx(b_path,current):
    path = "/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/cmake-build-debug/nex"
    print(f"{os.getpid()}: Running "+path+" "+b_path+" "+current)
    v = os.popen(path+" "+b_path+" "+current).read()
    print(f"{os.getpid()}: Done\n")
    v = v.split("\n")
    return v

def getElementsAndNodeCount(path):
    iFile = open(path)
    it = iFile.read()
    iFile.close()
    regNumNodes = r"\$Nodes\n(.*)\n"
    numNodes = re.findall(regNumNodes,it,re.MULTILINE)[0]
    regNumElem = r"\$Elements\n(.*)\n"
    numElem = re.findall(regNumElem,it,re.MULTILINE)[0]

    return numElem, numNodes

# writeRadius(15e-6, 0.2e-6)

# runGMSH()

# runTTH()

# i = getCurrent()

# runNoiseEx("/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/aVSn/output/loop_test.vtk",i)


# runSim()

## FIX D VARY R

In [5]:

b = 0.4e-6
lam = 0.4e-6
# lam = 24e-9

SPIN_DENS = 5e17
MU_B = 9.2740100783e-24
PHI_0 = 2.067833848e-15
MU_0 = 1.25663706212e-6 	
Analytic = lambda R, W : 2/3*MU_0**2*MU_B**2*SPIN_DENS*R/W*(np.log(2*b*W/(lam**2))/(2*np.pi)+0.27)*1/(PHI_0**2)

charLen = 3e-7
N = 400
# R = 15e-6
# W = 4e-6
z_off = 0.2e-6 # thickness
num_divs = 15
charLen_side = 0.4e-7

Rs = np.linspace(10e-6,15e-6,10)
Ws = np.linspace(7e-6,3e-6,10)




try:
    os.remove("resFIX_D_AreaErr.csv")
except:
    pass

res = open("resFIX_D_AreaErr.csv","w")
res.write("R,D,Analytic,Numerical\n")
it = 1
for R,W in tqdm(zip(Rs,Ws)):
    if it == 1:
        i = getCurrent(f"/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/CircularWasherSims_results/test{it}/test{it}/output/i.txt")
    else:
        i = getCurrent(f"/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/CircularWasherSims_results/test{it}/output/i.txt")
    print(i)
    if it == 1:
        msfn_numerical = runNoiseEx(f"/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/CircularWasherSims_results/test{it}/test{it}/output/B_loop_test.vtk",i)
    else:
        msfn_numerical = runNoiseEx(f"/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/CircularWasherSims_results/test{it}/output/B_loop_test.vtk",i)
    numerical_area = msfn_numerical[1]
    msfn_numerical = msfn_numerical[0]
    msfn_analytical = Analytic(R,W)
    # write result to file
    res.write(f"{R},{W},{msfn_analytical},{msfn_numerical}\n")
    it = it + 1


res.close()

0it [00:00, ?it/s]

1.0668150091e+00
6226: Running /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/cmake-build-debug/nex /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/CircularWasherSims_results/test1/test1/output/B_loop_test.vtk 1.0668150091e+00


1it [00:04,  4.42s/it]

6226: Done

8.3669533746e-01
6226: Running /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/cmake-build-debug/nex /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/CircularWasherSims_results/test2/output/B_loop_test.vtk 8.3669533746e-01


2it [00:07,  3.83s/it]

...

6226: Done

2.6346441970e-01
6226: Running /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/cmake-build-debug/nex /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/CircularWasherSims_results/test9/output/B_loop_test.vtk 2.6346441970e-01


9it [00:28,  2.90s/it]

6226: Done

1.1956856575e-01
6226: Running /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/cmake-build-debug/nex /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/CircularWasherSims_results/test10/output/B_loop_test.vtk 1.1956856575e-01


10it [00:32,  3.28s/it]

6226: Done

